In [1]:
import numpy as np
import pandas as pd

In [2]:
df_train = pd.read_csv('fake-news-pair-classification-challenge/train.csv')

In [3]:
df_train.head()

,id,tid1,tid2,title1_zh,title2_zh,title1_en,title2_en,label
0,0,0,1,2017养老保险又新增两项，农村老人人人可申领，你领到了吗,警方辟谣“鸟巢大会每人领5万” 仍有老人坚持进京,There are two new old-age insurance benefits f...,"Police disprove ""bird's nest congress each per...",unrelated
1,3,2,3,"""你不来深圳，早晚你儿子也要来""，不出10年深圳人均GDP将超香港",深圳GDP首超香港？深圳统计局辟谣：只是差距在缩小,"""If you do not come to Shenzhen, sooner or lat...",Shenzhen's GDP outstrips Hong Kong? Shenzhen S...,unrelated
2,1,2,4,"""你不来深圳，早晚你儿子也要来""，不出10年深圳人均GDP将超香港",GDP首超香港？深圳澄清：还差一点点……,"""If you do not come to Shenzhen, sooner or lat...",The GDP overtopped Hong Kong? Shenzhen clarifi...,unrelated
3,2,2,5,"""你不来深圳，早晚你儿子也要来""，不出10年深圳人均GDP将超香港",去年深圳GDP首超香港？深圳统计局辟谣：还差611亿,"""If you do not come to Shenzhen, sooner or lat...",Shenzhen's GDP topped Hong Kong last year? She...,unrelated
4,9,6,7,"""用大蒜鉴别地沟油的方法,怎么鉴别地沟油",吃了30年食用油才知道，一片大蒜轻松鉴别地沟油,"""How to discriminate oil from gutter oil by me...",It took 30 years of cooking oil to know that o...,agreed


In [4]:
empty_title = ((df_train['title1_zh'].isnull()) \
               | (df_train['title2_zh'].isnull()) \
               | (df_train['title2_zh'] == '') \
               | (df_train['title2_zh'] == 0))

In [5]:
print(len(df_train))

# Remove null rows
df_train = df_train[~empty_title]

print(len(df_train))

320552
320545


In [6]:
df_train.head().loc[3,'title1_zh']
df_train.title1_zh

0             2017养老保险又新增两项，农村老人人人可申领，你领到了吗
1         "你不来深圳，早晚你儿子也要来"，不出10年深圳人均GDP将超香港
2         "你不来深圳，早晚你儿子也要来"，不出10年深圳人均GDP将超香港
3         "你不来深圳，早晚你儿子也要来"，不出10年深圳人均GDP将超香港
4                      "用大蒜鉴别地沟油的方法,怎么鉴别地沟油
                        ...                
320547      萨拉赫人气爆棚!埃及总统大选未参选获百万选票 现任总统压力山大
320548      萨拉赫人气爆棚!埃及总统大选未参选获百万选票 现任总统压力山大
320549      萨拉赫人气爆棚!埃及总统大选未参选获百万选票 现任总统压力山大
320550      萨拉赫人气爆棚!埃及总统大选未参选获百万选票 现任总统压力山大
320551        萨达姆此项计划没有此国破坏的话，美国还会对伊拉克发动战争吗
Name: title1_zh, Length: 320545, dtype: object

In [7]:
# 去掉长度大于30的句子
MAX_LENGTH = 30
df_train = df_train[~df_train.title1_zh.apply(lambda x: (len(x) > MAX_LENGTH))]
df_train = df_train[~df_train.title2_zh.apply(lambda x: (len(x) > MAX_LENGTH))]

print(len(df_train))

265695


In [8]:
# 只取一瓢
SAMPLE_RATIO = 0.01

df_train = df_train.sample(frac = SAMPLE_RATIO)

In [9]:
print(len(df_train))

2657


In [10]:
# 重置index，只保存需要的标签，最后重命名
df_train.reset_index
df_train = df_train.loc[:, ['title1_zh', 'title2_zh', 'label']]
df_train.columns = ['text_a', 'text_b', 'label']

In [11]:
df_train.head()

,text_a,text_b,label
102861,别说你都知道，华为手机音量键还有这样的操作！,华为手机音量键还隐藏了这么多功能，玩手机这么久才发现，快试试,agreed
41004,一块陈皮能顶10副药，治疗膝盖疼效果惊人，家有膝盖疼的要存！,一颗洋葱顶10贴膏药，治疗膝盖疼效果惊人，家有膝盖疼的要存！,unrelated
11974,29岁释小龙回归，前女友嫌弃他，今事业辉煌前任离婚,能否与王菲谢霆锋一样何洁离婚后找释小龙,unrelated
5215,2017这几种袁大头市场才叫火爆，最贵的卖了598万我的天！,袁大头真品银元市场火爆 现在可以免费拍卖出手,unrelated
261192,王思聪放弃网红追腾讯千金，两人机场牵手相约吃饭？网友：我反对,王思聪预追马总千金？马化腾亲自辟谣！,disagreed


In [12]:
df_train.to_csv('train.tsv', sep = '\t', index = False)

In [13]:
df_train.label.value_counts() / len(df_train)

unrelated    0.684607
agreed       0.283779
disagreed    0.031615
Name: label, dtype: float64

In [14]:
df_train[df_train.label == 'disagreed']

,text_a,text_b,label
261192,王思聪放弃网红追腾讯千金，两人机场牵手相约吃饭？网友：我反对,王思聪预追马总千金？马化腾亲自辟谣！,disagreed
121153,吴亦凡被爆吸毒 13秒“摇头”影片疯传,吴亦凡吸毒视频疑造谣 工作室声明及时成公关范本,disagreed
58674,中超亚军主帅尚且下课，第六鲁能换帅亦成定局，新帅会是他吗？,鲁能换帅系谣言，那么换帅俱乐部会是哪几家呢？,disagreed
216510,最火的团队年终奖：王者荣耀团队年终奖：100个月的工资,王者荣耀：农药团队年终奖100个月工资，官方辟谣不属实,disagreed
232629,桃子和西瓜一起吃会致命？专家：我常一起吃,「读网」桃子西瓜一起吃不会丧命 不要相信谣言,disagreed
...,...,...,...
174764,张卫健吸毒被带走,张卫健辟谣什么 事件来龙去脉曝光一个举动被传吸毒,disagreed
113003,又一明星倒下？宋小宝被传患癌去世，不会吧,以讹传讹，宋小宝去世了！别逗了，宋小宝都出来辟谣了,disagreed
294025,网传“2017年免缴新农合医保费”，你当真了？,辟谣：2017年新农合不用缴费，已开始退款？假的！,disagreed
257810,独生子女将无法继承父母房产,独生子女将无法继承父母房产？谣言,disagreed


In [15]:
# 测试集数据
df_test = pd.read_csv('fake-news-pair-classification-challenge/test.csv')
df_test = df_test.loc[:, ['title1_zh', 'title2_zh', 'id']]
df_test.columns = ['text_a', 'text_b', 'label']
df_test.to_csv('test.tsv', sep = '\t', index = False)

In [16]:
len(df_test)

80126

In [17]:
import torch
from transformers import BertTokenizer
from IPython.display import clear_output
from torch.utils.data import Dataset

BERT:

![BERT sentence pair encoding](https://leemeng.tw/images/bert/practical_bert_encoding_for_pytorch.jpg)

tokens_tensor：代表識別每個 token 的索引值，用 tokenizer 轉換即可

segments_tensor：用來識別句子界限。第一句為 0，第二句則為 1。另外注意句子間的 [SEP] 為 0

masks_tensor：用來界定自注意力機制範圍。1 讓 BERT 關注該位置，0 則代表是 padding 不需關注

* [CLS]：在做分類任務時其最後一層的 repr. 會被視為整個輸入序列的 repr.
* [SEP]：有兩個句子的文本會被串接成一個輸入序列，並在兩句之間插入這個 token 以做區隔
* [UNK]：沒出現在 BERT 字典裡頭的字會被這個 token 取代
* [PAD]：zero padding 遮罩，將長度不一的輸入序列補齊方便做 batch 運算
* [MASK]：未知遮罩，僅在預訓練階段會用到

In [18]:
tokenizer = BertTokenizer.from_pretrained('bert-base-chinese')

text = '这是一个BERT的句子'
tokenizer.tokenize(text)

['这', '是', '一', '个', '[UNK]', '的', '句', '子']

In [19]:
class FakeNewsDataset(Dataset):
    def __init__(self, mode, tokenizer):
        assert mode in ['train', 'test']
        self.mode = mode
        file_name = mode + '.tsv'
        # 创建df，大数据需要使用iterator = True
        self.df = pd.read_csv(file_name, sep = '\t').fillna('')
        self.len = len(self.df)
        self.label_map = {'agreed':0, 'disagreed':1, 'unrelated':2}
        self.tokenizer = tokenizer
    
    # __getitem__可以通过trainset[5]的方式访问
    def __getitem__(self, idx):
        if self.mode == 'test':
            text_a, text_b = self.df.iloc[idx, :2].values
            label_tensor = None
        else:
            text_a, text_b, label = self.df.iloc[idx, :].values
            label_id = self.label_map[label] # 将agreed这种转换成1/2/3的格式
            label_tensor = torch.tensor(label_id)
        
        # 第一个句子，转化出来的都是单个的汉字
        word_pieces = ['[CLS]']
        tokens_a = self.tokenizer.tokenize(text_a)
        word_pieces += tokens_a + ['[SEP]']
        len_a = len(word_pieces)
        
        # 第二个句子
        tokens_b = self.tokenizer.tokenize(text_b)
        word_pieces += tokens_b + ['[SEP]']
        len_b = len(word_pieces) - len_a
        
        #将整个句子序列转换成index序列——汉字变成数字   
        ids = self.tokenizer.convert_tokens_to_ids(word_pieces)
        tokens_tensor = torch.tensor(ids)
        
        segments_tensor = torch.tensor([0]*len_a + [1]*len_b, dtype = torch.long)
        
        return (tokens_tensor, segments_tensor, label_tensor)
    
    def __len__(self):
        return self.len

In [20]:
trainset = FakeNewsDataset('train', tokenizer)

In [21]:
df_train.loc[:,'text_a']

102861            别说你都知道，华为手机音量键还有这样的操作！
41004     一块陈皮能顶10副药，治疗膝盖疼效果惊人，家有膝盖疼的要存！
11974          29岁释小龙回归，前女友嫌弃他，今事业辉煌前任离婚
5215      2017这几种袁大头市场才叫火爆，最贵的卖了598万我的天！
261192    王思聪放弃网红追腾讯千金，两人机场牵手相约吃饭？网友：我反对
                       ...              
167715                     岑溪市人民医院有人跳楼身亡
173044    建个猪场要十几亿，王健林怒骂：我的钱也不是自己印出来的（4）
290002    红包撤回速度能比得上抢红包的速度吗？微信准备加入红包撤回功能
201596                 收藏：28股，目标涨幅超过50%！
201120             支付宝提现开收手续费 免费攻略备受网友力捧
Name: text_a, Length: 2657, dtype: object

In [22]:
index = 0

text_a, text_b, label = trainset.df.iloc[index].values
tokens_tensor, segments_tensor, label_tensor = trainset[index]

tokens = tokenizer.convert_ids_to_tokens(tokens_tensor.tolist())
combined_text = ''.join(tokens)

print(f'''

Origin text_a: \n{text_a}
Origin text_b: \n{text_b}

Text tensors: \n{tokens_tensor}

Segments_tensor: \n{segments_tensor}

Label: {label_tensor}

Convert text: \n{combined_text}

''')



Origin text_a: 
别说你都知道，华为手机音量键还有这样的操作！
Origin text_b: 
华为手机音量键还隐藏了这么多功能，玩手机这么久才发现，快试试

Text tensors: 
tensor([ 101, 1166, 6432,  872, 6963, 4761, 6887, 8024, 1290,  711, 2797, 3322,
        7509, 7030, 7241, 6820, 3300, 6821, 3416, 4638, 3082,  868, 8013,  102,
        1290,  711, 2797, 3322, 7509, 7030, 7241, 6820, 7391, 5966,  749, 6821,
         720, 1914, 1216, 5543, 8024, 4381, 2797, 3322, 6821,  720,  719, 2798,
        1355, 4385, 8024, 2571, 6407, 6407,  102])

Segments_tensor: 
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1])

Label: 0

Convert text: 
[CLS]别说你都知道，华为手机音量键还有这样的操作！[SEP]华为手机音量键还隐藏了这么多功能，玩手机这么久才发现，快试试[SEP]




In [26]:
trainset[0]

(tensor([ 101, 1166, 6432,  872, 6963, 4761, 6887, 8024, 1290,  711, 2797, 3322,
         7509, 7030, 7241, 6820, 3300, 6821, 3416, 4638, 3082,  868, 8013,  102,
         1290,  711, 2797, 3322, 7509, 7030, 7241, 6820, 7391, 5966,  749, 6821,
          720, 1914, 1216, 5543, 8024, 4381, 2797, 3322, 6821,  720,  719, 2798,
         1355, 4385, 8024, 2571, 6407, 6407,  102]),
 tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1]),
 tensor(0))

In [22]:
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence

def create_mini_batch(samples):
    tokens_tensors = [s[0] for s in samples]
    segments_tensors = [s[1] for s in samples]
    
    # 区分了有无labels
    if samples[0][2] is not None:
        label_ids = torch.stack([s[2] for s in samples])
    else:
        label_ids = None
        
    # 通过pad_sequence 变成同样的长度
    tokens_tensors = pad_sequence(tokens_tensors, batch_first = True)
    segments_tensors = pad_sequence(segments_tensors, batch_first = True)
    
    # attention mask
    masks_tensors = torch.zeros(tokens_tensors.shape, dtype = torch.long)
    #Fills elements of self tensor with value where mask is True. 
    masks_tensors = masks_tensors.masked_fill(mask = tokens_tensors != 0,
                                            value = 1)
    
    return tokens_tensors, segments_tensors, masks_tensors, label_ids

In [23]:
BATCH_SIZE = 64
trainloader = DataLoader(trainset, batch_size = BATCH_SIZE,
                        collate_fn = create_mini_batch)

In [24]:
data = next(iter(trainloader))

tokens_tensors, segments_tensors, masks_tensors, label_ids = data

In [25]:
print(f"""
tokens_tensors.shape   = {tokens_tensors.shape} 
{tokens_tensors}
------------------------
segments_tensors.shape = {segments_tensors.shape}
{segments_tensors}
------------------------
masks_tensors.shape    = {masks_tensors.shape}
{masks_tensors}
------------------------
label_ids.shape        = {label_ids.shape}
{label_ids}
""")


tokens_tensors.shape   = torch.Size([64, 63]) 
tensor([[ 101, 2798, 4761,  ..., 4761, 6887,  102],
        [ 101, 3448, 3719,  ...,    0,    0,    0],
        [ 101, 1266,  776,  ...,    0,    0,    0],
        ...,
        [ 101,  126, 3299,  ...,    0,    0,    0],
        [ 101, 1283,  674,  ...,    0,    0,    0],
        [ 101, 2750, 1052,  ...,    0,    0,    0]])
------------------------
segments_tensors.shape = torch.Size([64, 63])
tensor([[0, 0, 0,  ..., 1, 1, 1],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]])
------------------------
masks_tensors.shape    = torch.Size([64, 63])
tensor([[1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])
------------------------
label_ids.shape        

In [28]:
# 在上游的Bert模型之下构建Linear classifier
# 记得在pycharm上看一眼model
from transformers import BertForSequenceClassification

PRETRAINED_MODEL_NAME = 'bert-base-chinese'
NUM_LABELS = 3

model = BertForSequenceClassification.from_pretrained(PRETRAINED_MODEL_NAME, num_labels = NUM_LABELS)

In [29]:
trainloader

Untitled.ipynb                          test.tsv
fake-news-pair-classification-challenge train.tsv
pytorch_train.tsv


In [38]:
def get_predictions(model, dataloader, compute_acc = False):
    # 初始化一些参数
    # 在no-grad下运算
    # output就是把相关三个参数输入到model当中去
    # 然后通过max取出预测值，然后跟实际y值比较
    
    with torch.no_grad():
        correct = 0
        total = 0
        predictions = None
        
        for data in dataloader:
            
            if next(model.parameters()).is_cuda:
                data = [t.to('cuda:0') for t in data if t is not None]
            
            tokens_tensors, segments_tensors, masks_tensors = data[:3]
            outputs = model(input_ids = tokens_tensors, token_type_ids = segments_tensors, attention_mask = masks_tensors)
            logits = outputs[0]
            _, pred = torch.max(logits, 1)
            
            if compute_acc:
                labels = data[3]
                total += labels.size(0)
                correct += (pred == labels).sum().item()
                
            if predictions is None:
                predictions = pred
            else:
                predictions = torch.cat((predictions, pred))
            
    if compute_acc:
        acc = correct / total
        return predictions, acc
    
    return predictions

In [39]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print('device: ', device)
model = model.to(device)
_, acc = get_predictions(model, trainloader, True)

device:  cpu


In [40]:
acc

0.6157320286036884

In [44]:
model.train()

optimizer = torch.optim.Adam(model.parameters(), lr = 1e-5)

EPOCHS = 6
for e in range(EPOCHS):
    running_loss = 0
    
    for data in trainloader:
        tokens_tensors, segments_tensors, masks_tensors, labels = [t.to(device) for t in data]
        optimizer.zero_grad()
        
        outputs = model(input_ids = tokens_tensors, token_type_ids = segments_tensors, 
                        attention_mask = masks_tensors, labels = labels)
        
        loss = outputs[0]
        
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        
    _, acc = get_predictions(model, trainloader, compute_acc = True)
    
    print('Epoch: {}\tRunning_loss:{.3f}\tAccurancy:{.3f}').format(epoch+1, running_loss, acc)

KeyboardInterrupt: 